## RE -> epsilon-NFA

(a+b)*abb
( recognize ( push , a recognize check len(stack) check stack[-1]+ a push ,+ push(just push) , b recognize check len(stack) check stack[-1]+ stack pop+ b push stack push+ , ) traverse until ( reveals check stack[index'('-1]+ check index'('0 pop all(just concat) , * check len(stack) write , a check len(stack)write, b write, b write

(b+a(aa*b)*b)* 
( recognize ( push, b recognize check len(stack) check stack[-1]+ b push , + push , a recognize check len(stack) check stack[-1]+ +pop 


(b+aa+ac+aaa+aac)*

(1+01)*00(0+1)*

(0+1)*011  


In [374]:
def isapt(c):
    assert len(c) == 1
    if ord(c) >= ord('a') and ord(c) <= ord('z') or ord(c) >= ord('A') and ord(c) <= ord('Z') or ord(c) >= ord('0') and ord(c) <= ord('9'):
        return True
    return False

In [375]:
def isfast(org, top):
    if org == '.' and top == '+':
        return True
    elif org == '*' and top != '*':
        return True
    return False
         

In [376]:
class node:
    def __init__(self, cn, cv, left=None, right=None):
        # cn : 0:+ 1:. 2:* 3:symbols
        self.cn = cn
        self.cv = cv
        self.left = None
        self.right = None

In [377]:
RE_ARR = ["(a+b)*abb", "(b+a(aa*b)*b)*" ,"(b+aa+ac+aaa+aac)*", "(1+01)*00(0+1)*" ,"(0+1)*011"]

In [378]:
RE = RE_ARR[0]
RE = "(a+b)*"
concatall = ""
str_arr = []

for i in range(len(RE)-1):
    str_arr.append(RE[i])
    if isapt(RE[i]):
        if isapt(RE[i+1]) or RE[i+1] == '(':
            str_arr.append('.')
    if (RE[i] == ')' or RE[i] == '*') and (RE[i+1] == '(' or isapt(RE[i+1])):
        str_arr.append('.')
str_arr.append(RE[-1])
concatall = "".join(str_arr)

print(concatall)

(a+b)*


In [379]:
stack = []
post_arr = []

for c in concatall:
    if isapt(c) or c == "*":
        post_arr.append(c)
    elif c == ")":
        while len(stack) > 0 and stack[-1] != "(":
            post_arr.append(stack.pop())
        stack.pop()
    elif c == "(":
        stack.append(c)
    elif len(stack) == 0 or stack[-1] == "(" or isfast(c, stack[-1]):
        
        stack.append(c)
    else:
        while len(stack) > 0 and stack[-1] != "(" and not isfast(c, stack[-1]):
            post_arr.append(stack.pop())
        stack.append(c)
while len(stack) > 0:
    post_arr.append(stack.pop())

postall = "".join(post_arr)
print(postall)

ab+*


In [380]:
stack = []
for c in postall:
        if c == "+":
            nd = node(0,c)
            nd.right = stack.pop()
            nd.left = stack.pop()
            stack.append(nd)
        elif c == ".":
            nd = node(1,c)
            nd.right = stack.pop()
            nd.left = stack.pop()
            stack.append(nd)
        elif c == "*":
            nd = node(2,c)
            nd.left = stack.pop() 
            stack.append(nd)
        elif c == "(" or c == ")":
            continue  
        else:
            stack.append(node(3,c))

In [381]:
class eNFA:
    def __init__(self):
        self.next = {}

In [382]:
def scan_plus(nd):
    start = eNFA()
    end = eNFA()

    left_eNFA = node_filter(nd.left)
    right_eNFA = node_filter(nd.right)

    start.next['ε'] = [left_eNFA[0], right_eNFA[0]]
    left_eNFA[1].next['ε'] = [end]
    right_eNFA[1].next['ε'] = [end]

    return start, end

def scan_dot(nd):
    left_nfa  = node_filter(nd.left)
    right_nfa = node_filter(nd.right)

    left_nfa[1].next['ε'] = [right_nfa[0]]
    return left_nfa[0], right_nfa[1]

def scan_star(nd):
    start = eNFA()
    end = eNFA()

    starred_nfa = node_filter(nd.left)

    start.next['ε'] = [starred_nfa[0], end]
    starred_nfa[1].next['ε'] = [starred_nfa[0], end]

    return start, end

def scan_symbol(nd):
    start = eNFA()
    end = eNFA()
    
    start.next[nd.cv] = [end]
    return start, end

In [383]:
def node_filter(nd):
    if nd.cn == 0:
        return scan_plus(nd)
    elif nd.cn == 1:
        return scan_dot(nd)
    elif nd.cn == 2:
        return scan_star(nd)
    elif nd.cn == 3:
        return scan_symbol(nd)
    else:
        print("error")

In [384]:
efas = node_filter(stack[0])

In [385]:
print(efas)
enfa = {}

(<__main__.eNFA object at 0x7f2ce9ff7a90>, <__main__.eNFA object at 0x7f2ce9ff7070>)


In [386]:
def sonbug_numbering(integer):
    assert integer < 1000, "too complicate RE"
    if integer > 99:
        return str(integer)
    elif integer > 9:
        return "0" + str(integer)
    else:
        return "00"+str(integer)

In [387]:
def arrange_transitions(State, States_history, symbols):

    if State in States_history:
        return

    States_history.append(State)

    for symbol in list(State.next):
        if symbol not in enfa['TerminalSet']:
            enfa['TerminalSet'].add(symbol)
        for ns in State.next[symbol]:
            if ns not in symbols:
                symbols[ns] = sorted(symbols.values())[-1] + 1
                q_state = "q" + sonbug_numbering(symbols[ns])
                enfa['StateSet'].add(q_state)
#enfa['DeltaFunctions'].append(["q" + sonbug_numbering(symbols[State]), symbol, "q" + sonbug_numbering(symbols[ns])])
            if not ("(q"+sonbug_numbering(symbols[State])+", "+symbol+")" in enfa['DeltaFunctions']):
                enfa['DeltaFunctions']["(q"+sonbug_numbering(symbols[State])+", "+symbol+")"] = set()
            enfa['DeltaFunctions']["(q"+sonbug_numbering(symbols[State])+", "+symbol+")"].add("q"+sonbug_numbering(symbols[ns]))      
        for ns in State.next[symbol]:
            arrange_transitions(ns, States_history, symbols)

In [388]:
enfa['StateSet'] = set()
enfa['TerminalSet'] = set()
enfa['DeltaFunctions'] = {}
enfa['StartState'] = set()
enfa['FinalStateSet'] = set()
q_0 = "q000"
enfa['StateSet'].add(q_0)
arrange_transitions(efas[0], [], {efas[0] : 0})
    

enfa["StartState"].add("q000")
for st in list(enfa["StateSet"]):
    count = 0
    for key,val in enfa['DeltaFunctions'].items():
        for value in list(val):
            if key[1:5] == st and value != st:
                count += 1
    if count == 0 and st not in enfa["FinalStateSet"]:
        enfa["FinalStateSet"].add(st)


In [389]:
print(enfa)

{'StateSet': {'q006', 'q002', 'q005', 'q001', 'q007', 'q003', 'q000', 'q004'}, 'TerminalSet': {'a', 'b', 'ε'}, 'DeltaFunctions': {'(q000, ε)': {'q001', 'q002'}, '(q001, ε)': {'q003', 'q004'}, '(q003, a)': {'q005'}, '(q005, ε)': {'q006'}, '(q006, ε)': {'q001', 'q002'}, '(q004, b)': {'q007'}, '(q007, ε)': {'q006'}}, 'StartState': {'q000'}, 'FinalStateSet': {'q002'}}
